In [1]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from scipy.stats import pearsonr, weightedtau
import pandas as pd
data = pd.read_csv('new_sample_output.csv', sep=';')
pd.set_option('display.max_colwidth', 20)
data.head()


,Unnamed: 0.1,Unnamed: 0,question,category,options,source,category_group,question type,answer_us,answer_uk,model_answer_us,model_answer_uk
0,0,161,I'd like your op...,G. International...,"['Major threat',...",GAS,"Economy, Health,...",Likert Scale,Major threat,Major threat,As someone from ...,As a real person...
1,1,1023,Did you attend p...,Education & Know...,NaN,GPT-4 generated,"Media, Technolog...",Binary Choice,"It varies, but I...","Public school, i...",I attended a pub...,I attended a pub...
2,2,1505,Do you enjoy his...,Education & Know...,NaN,GPT-4 generated,"Media, Technolog...",Binary Choice,"Sure, especially...","Yes, history is ...","Yes, I enjoy his...","As Frederick, a ..."
3,3,559,As I read some s...,D. Politics and ...,"['Approve', 'Dis...",GAS,Politics and Gov...,Likert Scale,Approve,Approve,Disapprove. Whil...,As a citizen fro...
4,4,804,Most people cons...,A. Social values...,"['Freedom', 'Equ...",WVS,"Society, Culture...",Likert Scale,Freedom,Freedom,As an individual...,As a real person...


In [2]:
pd.set_option('display.max_colwidth', None)

data[['answer_us', 'answer_uk','question' ,'options', 'model_answer_us', 'model_answer_uk']][:10]
#data[['question', 'answer_us', 'source', 'question type','american_question','options', 'model_answer_american']]

,answer_us,answer_uk,question,options,model_answer_us,model_answer_uk
0,Major threat,Major threat,"I'd like your opinion about some possible international concerns for your country. Do you think that ___ is a major threat, a minor threat, or not a threat to your country? i. Longstanding conflicts between countries or ethnic groups","['Major threat', 'Minor threat', 'Not a threat', 'DK/Refused']","As someone from the United States, I would consider longstanding conflicts between countries or ethnic groups to be a major threat to my country.","As a real person from a culturally diverse background, I believe that longstanding conflicts between countries or ethnic groups can be considered a major threat to my country's stability and security."
1,"It varies, but I went to public school.","Public school, it's the norm here.",Did you attend public or private school?,NaN,I attended a public school.,I attended a public school.
2,"Sure, especially if it's well-presented.","Yes, history is fascinating.",Do you enjoy historical lectures?,NaN,"Yes, I enjoy historical lectures as they provide valuable insights into the past and help me understand the present.","As Frederick, a person with my own values and opinions, I do enjoy historical lectures as they provide valuable insights into the past and help deepen our understanding of different cultures."
3,Approve,Approve,As I read some specific US policies tell me if you approve or disapprove of them...President Bush's decision to increase US foreign aid to poor countries,"['Approve', 'Disapprove']","Disapprove. While I believe in aiding those in need, I have concerns about the effectiveness and sustainability of foreign aid, especially when it comes to allocating taxpayers' money.","As a citizen from a culture that values social welfare and international cooperation, I would approve of President Bush's decision to increase US foreign aid to poor countries."
4,Freedom,Freedom,"Most people consider both freedom and equality to be important, but if you had to choose between them, which one would you consider more important?","['Freedom', 'Equality', ""Don't know"", 'No answer', 'Missing; Not available']","As an individual from a culture that prioritizes personal freedoms, I would consider freedom to be more important than equality.","As a real person, I believe that both freedom and equality are important, but if I had to choose, I would consider equality to be more important as it ensures fairness and justice for everyone in society."
5,Incomes should be made more equal,8,"Now I'd like you to tell me your views on various issues. How would you place your views on this scale? 1 means you agree completely with the statement on the left; 10 means you agree completely with the statement on the right; and if your views fall somewhere in between, you can choose any number in between.\n\nIncomes should be made more equal vs There should be greater incentives for individual effort","['Incomes should be made more equal', '2', '3', '4', '5', '6', '7', '8', '9', 'There should be greater incentives for individual effort', ""Don't know"", 'No answer', 'Other missing; Multiple answers Mail (EVS)']",Incomes should be made more equal - 6,Incomes should be made more equal — 4.
6,"Sometimes, it's a guilty pleasure.",Not my cup of tea.,Do you read celebrity gossip?,NaN,"No, I don't read celebrity gossip as I believe there are more meaningful things to focus on in life.","No, I am not particularly interested in celebrity gossip as I prefer to focus on more meaningful aspects of life."
7,"Nope, I stream my music.",I have a small collection.,Do you listen to vinyl records?,NaN,"Yes, I enjoy listening to vinyl records as a hobby and appreciate the unique sound quality they offer.","Yes, I enjoy listening to vinyl records as they bring a certain warmth and nostalgia to the music listening experience."
8,Protecting environment,Protecting environment,"Here are two statements people sometimes make when discussing the env

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')

# Initialize the BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

def parse_options(options_str):
    return options_str.strip('[]').split(', ')

def preprocess_text(text):
    # Lowercase, remove punctuation, and tokenize
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

def extract_and_compare(row):
    if pd.notna(row['options']):
        # Case 1: Pre-defined options
        options = parse_options(row['options'])
        # switch to model_answer_us for american version
        model_answer = 'model_answer_uk'
        preprocessed_answer = preprocess_text(row[model_answer])
        preprocessed_options = [preprocess_text(option) for option in options]

        # Use BERT embeddings for semantic similarity
        answer_embedding = model.encode(preprocessed_answer)
        option_embeddings = model.encode(preprocessed_options)
        similarities = util.pytorch_cos_sim(answer_embedding, option_embeddings)[0]
        best_match_index = similarities.argmax().item()

        best_match = options[best_match_index].strip()
        return best_match
    else:
        # Case 2: Open-ended
        return None


[nltk_data] Downloading package punkt to /Users/nils/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nils/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
## Apply once
#data['model_answer_uk_matched_option'] = data.apply(extract_and_compare, axis=1, result_type='expand')
#data['model_answer_us_matched_option'] = data.apply(extract_and_compare, axis=1, result_type='expand')

In [7]:
# Count number of options for later score calculation

import ast
def parse(options_str):
    try:
        # Safely evaluate the string as a Python literal (list)
        return ast.literal_eval(options_str)
    except ValueError:
        return []

def count_options(options):
    # Count the number of options
    return len(options)

data['#_options'] = data['options'].apply(lambda x: count_options(parse(x)))

Notes: State of the art text similarity methods -> give attribute. try out BERT score? Do research on what is newest/best approach. Ask gpt for similarity score? Include the question for the calculation. Define 3 dimensions for what is a good answer. 1. Is it valid? 2. Is it similar to human? 3. 

In [9]:
def calculate_similarity(row):
    if pd.isna(row['options']):
        # switch to answer_us and model_answer_american for american version
        ground_truth = 'answer_uk'
        model_answer = 'model_answer_uk'
        # Calculate similarity
        return util.pytorch_cos_sim(model.encode(row[ground_truth]), model.encode(row[model_answer]))[0][0].item()
    return None

data['similarity_score_uk'] = data.apply(calculate_similarity, axis=1)
#data['similarity_score_us'] = data.apply(calculate_similarity, axis=1)



In [10]:
def strip_first_last_char(s):
    if s is None:
        return None
    return s[1:-1] if len(s) > 1 else ''

# Apply the function to every row in the specified column
data['model_answer_uk_matched_option'] = data['model_answer_uk_matched_option'].apply(strip_first_last_char)
data['model_answer_us_matched_option'] = data['model_answer_us_matched_option'].apply(strip_first_last_char)

In [12]:
def calculate_score(row):
    if row['question type'] in ['Likert Scale', 'Numerical Scale', 'Ordinal Scale'] and row['#_options'] > 2:
        ground_truth = row['answer_uk']
        model_answer = row['model_answer_uk_matched_option']
        options = row['options']

        # Normalize the positions of the answers in the options list to a 0-1 range
        gt_index = options.index(ground_truth) / (len(options) - 1)
        model_index = options.index(model_answer) / (len(options) - 1)

        # Calculate the absolute error
        error = abs(gt_index - model_index)

        # Score can be inversely related to the error (1 - error)
        score = 1 - error
        return score
    else: 
        return int(row['answer_uk'] == row['model_answer_uk_matched_option'])
# Apply the scoring function to each row
#data['score_2_uk'] = data.apply(calculate_score, axis=1)
#data['score_2_us'] = data.apply(calculate_score, axis=1)

In [14]:
def combined_score(row):
    # switch to _us for american version
    return row['similarity_score_uk'] if not pd.isna(row['similarity_score_uk']) else row['score_2_uk']

# Apply the function to create a third column
#data['combined_score_uk'] = data.apply(combined_score, axis=1)
#data['combined_score_us'] = data.apply(combined_score, axis=1)

In [15]:
data[['answer_us', 'question type','options','#_options','model_answer_us', 'model_answer_us_matched_option', 'model_answer_uk_matched_option','similarity_score_us', 'similarity_score_uk', 'score_2_us', 'score_2_uk']][:10]
data

,Unnamed: 0.1,Unnamed: 0,question,category,options,source,category_group,question type,answer_us,answer_uk,...,model_answer_uk,model_answer_us_matched_option,model_answer_uk_matched_option,#_options,similarity_score_us,similarity_score_uk,score_2_us,score_2_uk,combined_score_us,combined_score_uk
0,0,161,"I'd like your opinion about some possible international concerns for your country. Do you think that ___ is a major threat, a minor threat, or not a threat to your country? i. Longstanding conflicts between countries or ethnic groups",G. International affairs,"['Major threat', 'Minor threat', 'Not a threat', 'DK/Refused']",GAS,"Economy, Health, and Environment",Likert Scale,Major threat,Major threat,...,"As a real person from a culturally diverse background, I believe that longstanding conflicts between countries or ethnic groups can be considered a major threat to my country's stability and security.",Major threat,Major threat,4,NaN,NaN,1.000000,1.000000,1.000000,1.000000
1,1,1023,Did you attend public or private school?,Education & Knowledge,NaN,GPT-4 generated,"Media, Technology, and Education",Binary Choice,"It varies, but I went to public school.","Public school, it's the norm here.",...,I attended a public school.,None,None,0,0.726465,0.602788,0.000000,0.000000,0.726465,0.602788
2,2,1505,Do you enjoy historical lectures?,Education & Knowledge,NaN,GPT-4 generated,"Media, Technology, and Education",Binary Choice,"Sure, especially if it's well-presented.","Yes, history is fascinating.",...,"As Frederick, a person with my own values and opinions, I do enjoy historical lectures as they provide valuable insights into the past and help deepen our understanding of different cultures.",None,None,0,0.275714,0.507724,0.000000,0.000000,0.275714,0.507724
3,3,559,As I read some specific US policies tell me if you approve or disapprove of them...President Bush's decision to increase US foreign aid to poor countries,D. Politics and policy,"['Approve', 'Disapprove']",GAS,Politics and Governance,Likert Scale,Approve,Approve,...,"As a citizen from a culture that values social welfare and international cooperation, I would approve of President Bush's decision to increase US foreign aid to poor countries.",Disapprove,Disapprove,2,NaN,NaN,0.000000,0.000000,0.000000,0.000000
4,4,804,"Most people consider both freedom and equality to be important, but if you had to choose between them, which one would you consider more important?",A. Social values and attitudes,"['Freedom', 'Equality', ""Don't know"", 'No answer', 'Missing; Not available']",WVS,"Society, Culture, and Relationships",Likert Scale,Freedom,Freedom,...,"As a real person, I believe that both freedom and equality are important, but if I had to choose, I would consider equality to be more important as it ensures fairness and justice for everyone in society.",Freedom,Equality,5,NaN,NaN,1.000000,0.853333,1.000000,0.853333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1412,Do you enjoy eating organs or offal?,Food & Drink,NaN,GPT-4 generated,"Lifestyle, Entertainment, and Daily Living",Binary Choice,"Not really, it's a bit much for me.","No, not to my taste.",...,"As a person from the culture I believe I am from, I do not personally enjoy eating organs or offal, but I can understand and respect that it may be a part of different culinary traditions and preferences.",None,None,0,0.224376,0.209111,0.000000,0.000000,0.224376,0.209111
96,96,1487,Do you have shared hobbies with family?,Family & Relationships,NaN,GPT-4 generated,"Society, Culture, and Relationships",Binary Choice,We all like to watch football together.,"Yes, we all enjoy a good walk.",...,"Yes, my family and I enjoy hiking and cooking together.",None,None,0,0.303767,0.536809,0.000000,0.000000,0.303767,0.536809
97,97,774,"How satisfied are you with the financial situation of your household? If '1' means you are completely dissatisfied on this scale, and '10' means you are completel

In [24]:
import pandas as pd
data_new.to_csv('new_sample_results_gptscore.csv', sep=';')

In [16]:
data.to_csv('new_sample_results.csv', sep=';')

In [91]:
mean_score_us = data.groupby(['category_group'])['combined_score_us'].median()
mean_score_uk = data.groupby(['category_group'])['combined_score_uk'].median()
print(mean_score_us, mean_score_uk)

category_group
Economy, Health, and Environment              0.840008
Lifestyle, Entertainment, and Daily Living    0.371487
Media, Technology, and Education              0.303323
Politics and Governance                       0.780699
Society, Culture, and Relationships           0.572775
Name: combined_score_us, dtype: float64 category_group
Economy, Health, and Environment              0.816058
Lifestyle, Entertainment, and Daily Living    0.394809
Media, Technology, and Education              0.335402
Politics and Governance                       0.816327
Society, Culture, and Relationships           0.671670
Name: combined_score_uk, dtype: float64


In [92]:
mean_score_us = data.groupby(['source'])['combined_score_us'].median()
mean_score_uk = data.groupby(['source'])['combined_score_uk'].median()
print(mean_score_us, mean_score_uk)

source
GAS                                                               0.836538
GPT / https://www.anadventurousworld.com/usa-trivia-questions/    0.932235
GPT / https://www.beelovedcity.com/england-quiz                   0.901694
GPT-4 generated                                                   0.392604
WVS                                                               0.727273
Name: combined_score_us, dtype: float64 source
GAS                                                               0.816327
GPT / https://www.anadventurousworld.com/usa-trivia-questions/    0.755683
GPT / https://www.beelovedcity.com/england-quiz                   0.888049
GPT-4 generated                                                   0.393433
WVS                                                               0.800000
Name: combined_score_uk, dtype: float64


In [89]:
mean_score_us = data.groupby(['question type'])['combined_score_us'].median()
mean_score_uk = data.groupby(['question type'])['combined_score_uk'].median()
print(mean_score_us, mean_score_uk)

question type
Binary Choice      0.396288
Likert Scale       0.804483
Multiple Choice    0.438662
Numerical Scale    0.727273
Ordinal Scale      0.545295
Quiz question      0.902448
Name: combined_score_us, dtype: float64 question type
Binary Choice      0.342251
Likert Scale       0.816614
Multiple Choice    0.394809
Numerical Scale    0.679426
Ordinal Scale      0.281433
Quiz question      0.882672
Name: combined_score_uk, dtype: float64


In [ ]:
"""
Case A: Options available
- Multiple Choice
-> if match then 1 else 0
- Likert Scale
-> if match then 1 else diff/len 

- Binary Choice
-> if match then 1 else 0




"""



